## Training a model offline for the task

### Importing libraries needed

In [68]:
import numpy as np
import torch
import torch.nn as nn
import collections 
import random
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [69]:
def build_dictionary(dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    text_file.close()
    return full_dictionary

full_dictionary_location = "words_250000_train.txt"
full_dictionary = build_dictionary(full_dictionary_location)       
full_dictionary_common_letter_sorted = collections.Counter("".join(full_dictionary)).most_common()

In [70]:
len(full_dictionary)

227300

### Offline Hangman player for training

In [71]:
class Hangman_offline:
    def __init__(self, word, model, lives=6):
        self.word = word
        self.model = model
        self.lives = lives
        self.guessed = set()
        self.remaining = set(ord(i)-97 for i in self.word)
        self.prev_states = []
        self.guesses_made = []
        self.probabilities = []
        return

    def OH_current_state(self):
        indices = [(ord(i)-97 if ord(i)-97 in self.guessed else 26) for i in self.word]
        one_hot_state = torch.zeros((len(self.word),27), dtype=torch.float32)
        for i,j in enumerate(indices): 
            one_hot_state[i,j] = 1
        return one_hot_state

    def OH_guess(self, guess):
        one_hot_guess = torch.zeros(26, dtype=torch.float32)
        one_hot_guess[guess] = 1
        return one_hot_guess

    def ID_guessed_letters(self):
        id_guesses = torch.zeros(26, dtype=torch.float32)
        for i in self.guessed:
            id_guesses[i] = 1.0
        return id_guesses

    def correct_probabilities(self):
        probabilities = torch.zeros(26, dtype=torch.float32)
        for i in self.remaining:
            probabilities[i] = 1.0
        probabilities /= probabilities.sum()
        return probabilities            

    def actions_on_guess(self, guess):
        self.prev_states.append(self.OH_current_state())
        self.guesses_made.append(guess)
        self.guessed.add(guess)
        
        answer_probability = self.correct_probabilities()
        self.probabilities.append(answer_probability)
        
        if guess in self.remaining:
            self.remaining.remove(guess)
        if self.probabilities[-1][guess] < 1e-5:
            self.lives -= 1
            
        return

    def run_game(self):
                
        while self.lives and self.remaining:
            masked_word = self.OH_current_state().to(device)
            prev_guesses = self.ID_guessed_letters().to(device)

            model.eval()
            model.to(device)
            
            output = self.model(masked_word.unsqueeze(0), prev_guesses.unsqueeze(0))
            guess = torch.argmax(output).item()

            while prev_guesses[guess]:
                output[0][guess] = -float('inf')
                guess = torch.argmax(output).item()
                

            self.actions_on_guess(guess)
            
            # print("Shape of probs", np.array(self.probabilities).shape)
            # for i in self.probabilities:
            #     print(i.shape)
            
        return (torch.from_numpy(np.array(self.prev_states)), 
                torch.from_numpy(np.array([self.OH_guess(i) for i in self.guesses_made])), 
                torch.from_numpy(np.array(self.probabilities)))

    def plays_view(self):
        print(f"Actual answer : {self.word}")
        for i in range(len(self.guesses_made)):
            view = ''.join(['-' if j==26 else chr(j+97) for j in self.prev_states[i].argmax(axis=1)])
            print(f"Guessed {chr(self.guesses_made[i]+97)} when showed {view}")
        print("Game Won!" if self.lives else "Game Lost :(")

    def evaluate_performance(self):
        success = self.lives > 0
        letters = set(self.word)
        correct_guesses = len(letters) - len(self.remaining)
        incorrect_guesses = len(self.guesses_made) - correct_guesses
        return (success, correct_guesses, incorrect_guesses, letters)

### Creating the model

In [282]:
class BI_LSTM_model(nn.Module):
    
    def __init__(self, size_word, size_guessed, hidden_size_lstm, out_size):
        super(BI_LSTM_model, self).__init__()
        self.lstm = nn.LSTM(size_word, hidden_size_lstm, num_layers=3, bidirectional=True, dropout=0.2, batch_first=True)
        self.lin1 = nn.Linear(hidden_size_lstm * 2 + size_guessed, 128)  # hidden_size * 2 because of bidirectional
        self.lin2 = nn.Linear(128, out_size)
        self.dropout = nn.Dropout(0.4)
    
    def forward(self, x1, x2):
        lstm_out, _ = self.lstm(x1)
        lstm_out = lstm_out[:, -1, :]  # Take output of the last time step
        x1 = self.dropout(lstm_out)
        x = self.lin1(torch.cat((x1,x2), dim=1))
        x = self.dropout(x)
        x = self.lin2(x)
        return x

size_word = 27
size_guessed = 26
hidden_size_lstm = 256
out_size = 26

model = BI_LSTM_model(size_word, size_guessed, hidden_size_lstm, out_size)
checkpoint = torch.load('./models/241.pt', weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

BI_LSTM_model(
  (lstm): LSTM(27, 256, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (lin1): Linear(in_features=538, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=26, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [283]:
word = 'microsoft'

player = Hangman_offline(word, model)
#Sample run
masked_word = player.OH_current_state()
prev_guesses = player.ID_guessed_letters()

output = model(masked_word.unsqueeze(0).to(device), prev_guesses.unsqueeze(0).to(device))
print("guess letter : {}".format(chr(torch.argmax(output).item()+97)))

guess letter : e


In [284]:
player.run_game()
player.plays_view()

Actual answer : microsoft
Guessed e when showed ---------
Guessed a when showed ---------
Guessed i when showed ---------
Guessed s when showed -i-------
Guessed r when showed -i---s---
Guessed t when showed -i-r-s---
Guessed o when showed -i-r-s--t
Guessed n when showed -i-roso-t
Guessed l when showed -i-roso-t
Guessed c when showed -i-roso-t
Guessed d when showed -icroso-t
Guessed u when showed -icroso-t
Game Lost :(


In [285]:
correct = 0
played = 0

In [288]:
this_correct = 0
this_played = 0

start = time.time()
for i,word in enumerate(random.sample(full_dictionary,200)):
    if i and i%10000 == 0: print(f"playing {i}-th game at {time.time()-start}")
    player = Hangman_offline(word, model)
    player.run_game()
    success, _, _, _ = player.evaluate_performance()
    played += 1
    this_played += 1
    if success:
        correct += 1
        this_correct +=1

this_accuracy = this_correct / this_played

In [289]:
accuracy = correct / played
print(f"Last  Accuracy: {this_accuracy*100}%, won {this_correct} out of {this_played}")
print(f"Total Accuracy: {accuracy*100}%, won {correct} out of {played}")

Last  Accuracy: 10.5%, won 21 out of 200
Total Accuracy: 11.5%, won 46 out of 400
